# PageRank

## Обработка данных

In [1]:
%pip install scipy
%pip install numpy
%pip install pandas
%pip install networkx
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [2]:
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.model_selection import train_test_split
from typing import Tuple, Mapping, Any

In [3]:
df = pd.read_csv("res\\rating.csv")

In [4]:
df['rating'].describe()

count    2.000026e+07
mean     3.525529e+00
std      1.051989e+00
min      5.000000e-01
25%      3.000000e+00
50%      3.500000e+00
75%      4.000000e+00
max      5.000000e+00
Name: rating, dtype: float64

In [7]:
def split_by_target_column_value(df: pd.DataFrame, test_size: float, target_column: str, target_value: Any,
                                 random_state: int = 42) -> Tuple[pd.DataFrame, pd.Series, pd.DataFrame, pd.Series]:
    """
    Разделяет DataFrame на обучающий и тестовый наборы на основе определенного значения целевого столбца.
    Выдает в качестве тестового набора test_size от набора, в котором значение в target_column == target_value
    В тренировочном все остальное
    Параметры:
        df (pd.DataFrame): DataFrame для разделения.
        test_size: Пропорция данных, которые следует включить в тестовый набор.
        target_column (str): Название целевого столбца для разделения.
        target_value (Any): Значение целевого столбца, используемое для разделения.
        random_state (int): Зерно для генератора случайных чисел.

    Возвращает:
        Tuple[pd.DataFrame, pd.Series, pd.DataFrame, pd.Series]: Кортеж, содержащий данные для обучения,
        метки обучения, данные для тестирования и метки тестирования.
    """
    grouped = df.groupby(by=target_column)
    train_df, test_df = train_test_split(grouped.get_group(target_value), test_size=test_size,
                                         random_state=random_state)
    train = pd.concat([group for ind, group in grouped if ind != target_value], ignore_index=True)
    train = pd.concat([train_df, train], ignore_index=True)
    train_y = train_df[target_column]
    test_y = test_df[target_column]
    return train, train_y, test_df, test_y



In [8]:
train1, train1_y, test1, test1_y = split_by_target_column_value(df, test_size=0.5, target_column='userId', target_value=1)

## Реализация PageRank

In [9]:
def get_range(train: pd.DataFrame, personalization: Mapping,
               source: str, target: str, edge_attr: str) -> pd.DataFrame:
    """
    Вычисляет PageRank на основе заданного DataFrame и возвращает DataFrame с результатами.
    В возвращаемых значении присутствуют данные из personalization
    Параметры:
        train (pd.DataFrame): DataFrame для вычисления PageRank.
        personalization (Mapping): Словарь с персонализацией для PageRank.
        source (str): Название столбца, содержащего исходные вершины ребер.
        target (str): Название столбца, содержащего целевые вершины ребер.
        edge_attr (str): Название столбца, содержащего атрибуты ребер.

    Возвращает:
        pd.DataFrame: DataFrame с результатами вычисления PageRank.
    """
    # Создание DataFrame с ребрами
    edges = train[[source, target, edge_attr]]
    edges[source] = train[source].apply(lambda x: (x, 'source'))
    edges[target] = train[target].apply(lambda x: (x, 'target'))
    
    # Создание графа из DataFrame с ребрами
    G = nx.from_pandas_edgelist(edges, source=source, target=target, edge_attr=edge_attr)
    
    # Преобразование персонализации в формат, подходящий для PageRank
    new_personalization = {(k, 'source'): v for k, v in personalization.items()}
    
    # Вычисление PageRank
    pagerank = nx.pagerank(G, personalization=new_personalization)
    
    # Формирование DataFrame с результатами PageRank
    target_rank_generator = ((node[0], rank) for node, rank in pagerank.items() if node[1] == 'target')
    return pd.DataFrame(target_rank_generator, columns=[target, 'pagerank'])


In [10]:
prediction_movie_for_user1 = get_range(train=train1, personalization={1: 1}, source='userId', target='movieId',
                                            edge_attr='rating')
prediction_movie_for_user1_without_famous = prediction_movie_for_user1[~prediction_movie_for_user1.isin(train1_y)]

C:\Users\odo20\AppData\Local\Temp\ipykernel_8532\30100332.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edges[source] = train[source].apply(lambda x: (x, 'source'))
C:\Users\odo20\AppData\Local\Temp\ipykernel_8532\30100332.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edges[target] = train[target].apply(lambda x: (x, 'target'))


In [11]:
pred = prediction_movie_for_user1_without_famous.sort_values(by='pagerank', ascending=False).head(len(test1_y))


In [12]:
def accuracy_without_order(true_labels, predicted_labels):
    # Преобразование списков в множества
    true_set = set(true_labels)
    pred_set = set(predicted_labels)
    
    # Проверка на равенство множеств
    accuracy = len(true_set.intersection(pred_set)) / len(true_set.union(pred_set))
    
    return accuracy

In [13]:
print(accuracy_without_order(test1_y, pred))

0.0


In [14]:
movie_df = pd.read_csv('res/movie.csv')

In [15]:
prediction_moviename = pd.merge(movie_df, pred, on='movieId', how='inner')
prediction_moviename.sort_values(by='pagerank', ascending=False).head()

,movieId,title,genres,pagerank
8,593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,0.002580
1,47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,0.002226
64,4993,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy,0.002182
13,1214,Alien (1979),Horror|Sci-Fi,0.002087
59,4306,Shrek (2001),Adventure|Animation|Children|Comedy|Fantasy|Ro...,0.002072


In [18]:
test_moviename = pd.merge(movie_df, test1, on='movieId', how='inner')
test_moviename.sort_values(by='rating', ascending=False).head()

,movieId,title,genres,userId,rating,timestamp
73,5952,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy,1,5.0,2005-04-02 23:30:19
87,8636,Spider-Man 2 (2004),Action|Adventure|Sci-Fi|IMAX,1,4.5,2005-04-02 23:44:53
21,1198,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure,1,4.5,2005-04-02 23:30:24
20,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi,1,4.5,2005-04-02 23:32:22
54,3499,Misery (1990),Drama|Horror|Thriller,1,4.0,2005-04-02 23:35:18


In [19]:
len(pred)

88

In [20]:
len(test1_y)

88